In [ ]:
#install necessary dependencies
!pip install flask pyngrok requests --quiet

#setting up ngrok authentication
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz --quiet
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!./ngrok config add-authtoken 2q1QOym4phRHlm9QPGJuJfBY1zo_ye63SKnsh7bsNN6xFMBB

ngrok
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
#import required libraries
from flask import Flask, request, jsonify
from pyngrok import ngrok
from cryptography.fernet import Fernet

In [ ]:
#initiate the flask app
app = Flask(__name__)

#root route
@app.route("/")
def home():
    return "Secure Messenger is live!"

In [ ]:
#secret key generation
SECRET_KEY = Fernet.generate_key()
cipher = Fernet(SECRET_KEY)

In [ ]:
#message storing in memory
stored_message = None

In [ ]:
#sending message
@app.route('/send_message', methods=['POST'])
def send_message():
    global stored_message
    data = request.json
    if 'message' not in data:
        return jsonify({"error": "Message field is required"}), 400

    #message encryption
    encrypted_message = cipher.encrypt(data['message'].encode())
    stored_message = encrypted_message
    return jsonify({"message": "Message encrypted and stored successfully"}), 200

In [ ]:
#receiving message
@app.route('/get_message', methods=['GET'])
def get_message():
    global stored_message
    if not stored_message:
        return jsonify({"error": "No message found"}), 404

    #message decryption
    decrypted_message = cipher.decrypt(stored_message).decode()
    return jsonify({"message": decrypted_message}), 200

In [ ]:
#opening public url using ngrok
if __name__ == "__main__":
    public_url = ngrok.connect(5000).public_url
    print(f"Public URL: {public_url}")
    app.run()

Public URL: https://4b35-34-83-95-99.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Dec/2024 20:01:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Dec/2024 20:01:37] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Dec/2024 20:02:38] "POST / HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [12/Dec/2024 20:03:26] "POST /send_message HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Dec/2024 20:04:00] "GET /get_message HTTP/1.1" 200 -
